In [1]:
from benchmarks import power_2samp_sample
from hyppo.independence import CCA, Dcorr, HHG, Hsic, RV, MGC
from hyppo.sims import *


import sys, os
import multiprocessing as mp
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt

In [2]:
import hyppo
import numpy as np
from numba import njit
from hyppo.independence.base import IndependenceTest
from hyppo._utils import perm_test
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import copy 
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.tree._classes import DecisionTreeClassifier
from joblib import Parallel, delayed
from scipy.stats import entropy, multivariate_normal
from scipy.integrate import nquad

import sys
sys.executable
sys.path
sys.path.append('C:\\Users\\siptest\\AppData\\Roaming\\Python\\Python36\\Scripts')
sys.path.append('C:\\Users\\siptest\\Desktop\\R-3.6.2\\bin\\x64')

In [3]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[3:])

In [13]:
def uf(X, y, n_estimators = 300, max_samples = .4, base = np.exp(1), kappa = 3):
#def uf(X, y, n_estimators = 300, max_samples = .4, base = 2, kappa = 3):        
        # Build forest with default parameters.
        model = BaggingClassifier(DecisionTreeClassifier(), 
                                  n_estimators=n_estimators, 
                                  max_samples=max_samples, 
                                  bootstrap=False)
        model.fit(X, y)
        n = X.shape[0]
        K = model.n_classes_
        _, y = np.unique(y, return_inverse=True)
        
        cond_entropy = 0
        for tree_idx, tree in enumerate(model):
            # Find the indices of the training set used for partition.
            sampled_indices = model.estimators_samples_[tree_idx]
            unsampled_indices = np.delete(np.arange(0,n), sampled_indices)
            
            # Randomly split the rest into voting and evaluation.
            total_unsampled = len(unsampled_indices)
            np.random.shuffle(unsampled_indices)
            vote_indices = unsampled_indices[:total_unsampled//2]
            eval_indices = unsampled_indices[total_unsampled//2:]
            
            # Store the posterior in a num_nodes-by-num_classes matrix.
            # Posteriors in non-leaf cells will be zero everywhere
            # and later changed to uniform.
            node_counts = tree.tree_.n_node_samples
            class_counts = np.zeros((len(node_counts), K))
            est_nodes = tree.apply(X[vote_indices])
            est_classes = y[vote_indices]
            for i in range(len(est_nodes)):
                class_counts[est_nodes[i], est_classes[i]] += 1
            
            row_sums = class_counts.sum(axis=1) # Total number of estimation points in each leaf.
            row_sums[row_sums == 0] = 1 # Avoid divide by zero.
            class_probs = class_counts / row_sums[:, None]
            
            # Make the nodes that have no estimation indices uniform.
            # This includes non-leaf nodes, but that will not affect the estimate.
            class_probs[np.argwhere(class_probs.sum(axis = 1) == 0)] = [1 / K]*K
            
            # Apply finite sample correction and renormalize.
            where_0 = np.argwhere(class_probs == 0)
            for elem in where_0:
                class_probs[elem[0], elem[1]] = 1 / (kappa*class_counts.sum(axis = 1)[elem[0]])
            row_sums = class_probs.sum(axis=1)
            class_probs = class_probs / row_sums[:, None]
            
            # Place evaluation points in their corresponding leaf node.
            # Store evaluation posterior in a num_eval-by-num_class matrix.
            eval_class_probs = class_probs[tree.apply(X[eval_indices])]
            # eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
            
            #print(eval_class_probs)
            
            eval_entropies = [entropy(posterior) for posterior in eval_class_probs]
            
            #print(eval_entropies)
            
            cond_entropy += np.mean(eval_entropies)
    
        print(cond_entropy)  
        return cond_entropy / n_estimators

In [5]:
MAX_SAMPLE_SIZE = 100
#STEP_SIZE = 5
STEP_SIZE = 20
SAMP_SIZES = range(20, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
#SAMP_SIZES = range(5, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
POWER_REPS = 20

In [15]:
#every time generate data set, call uf to get test stat = conditional entropy for number of power reps 
#null distribution: use y vector, numpy.random.binomial(lengthof(y), sum(y)/lengthof(y)), call entropy of vector, put into list

#pvalue - num random entropies larger than UF / 1000 (or length of reps from null dist)

#power is prob is that when alt is true, we reject null hypothesis, prob we see p value < 0.05 for example
#to calculate: 10-100 number of pvalues, proportion of p-values less than 0.05,

#in simulation, noise should be True 
#n = 100, p =1, noise = TRUE
#200 x 2 is x
#0s or 1s that tell you whether from first multimodal or shifted multimodal 

#1000x entropy(numpy.random.binomial(200, 0.5)) add to list 

from hyppo.sims import ksample_sim
from hyppo.ksample._utils import k_sample_transform

nullDist = []
success= 0
for i in range(1000): 
    temp = []
    #nullDist.append(entropy(np.random.binomial(1, 0.5, size = (200,))))
    #print(np.random.binomial(1, 0.5, size = 200))
    success = np.random.binomial(10000, 0.5)
    temp.append(success/10000)
    temp.append((10000-success)/10000)
    #print(temp)
    #print(entropy(temp, base=2))
    nullDist.append(entropy(temp))
print(nullDist)

power = []

for x in SAMP_SIZES: 
    pvalues = []
    for p in range(POWER_REPS): 
        print(p)
        #matrix1, matrix2 = ksample_sim.trans_2samp(multimodal_independence, 100, 1, noise = False, trans = 0.3)
        matrix1, matrix2 = ksample_sim.trans_2samp(multimodal_independence, 500, 1, noise = False, trans = 0.3)
        
        x, y = k_sample_transform([matrix1, matrix2])
        ce = uf(x, y)
        print(ce)
        pvalue = (nullDist >= ce).sum() / 1000
        print(pvalue)
        pvalues.append(pvalue)
    power.append((pvalues >= 0.05).sum() / POWER_REPS)
    
np.savetxt('C:/Users/siptest/Desktop/hyppo/benchmarks/2samp_vs_samplesize/UF_MultiModal_NotNull.csv',
               power, delimiter=',')
        
        
        
#from hyppo.sims import ksample_sim
#matrix1, matrix2 = ksample_sim.trans_2samp(multimodal_independence, 100, 1, noise=True, trans=0.5)

#from hyppo.ksample._utils import k_sample_transform
#x, y = k_sample_transform([matrix1, matrix2])

#ce = uf(x, y)
#calculate p-value, add to list of pvalues, after power rep, see how many are less than 0.05 

#sample size, power = number less than 0.05, vector of power, plot 
#list of powers that append every sample size 
#plot that to generate curve 

#power curve should look like growth, should start at something small, should end up being 1 

#generate data using trans/rot 2 samp 

#POWER-REPS = 20 at least 

#for i in POWER_REPS: 
    #compute p-value and store into list
    
#find how many p-values < 0.05 
#for every sample size have single number, plot sample size vs numbers, as sample size >, power > 
#sample szie is outside loop - how smooth 

#run in 2 cases: null case (no difference between 2 cases): translation value is 0 (trans = 0 in trans_2_samp)
#run when translation value is 0.5 start there 

#2 csv's, one for null and one for alternative, numpy save
#also tell file if null or alternative 


[0.6931029999092981, 0.6930600580298041, 0.6931066800131775, 0.6931010998521303, 0.6931102001040916, 0.6931119001450408, 0.6931468605599111, 0.6931468605599111, 0.6930751788318458, 0.6926603815637292, 0.6930844592486134, 0.6930971797265786, 0.6931291804519437, 0.6930971797265786, 0.6931346805078614, 0.6931151802186033, 0.6930491773583609, 0.6930026735990573, 0.6931468605599111, 0.6931443005571805, 0.6931383605340142, 0.6931240603817639, 0.6931254004018198, 0.6930930995850211, 0.6930821991524053, 0.6930909995078338, 0.6931471605599452, 0.6931471605599452, 0.6930463571714478, 0.6931315004779901, 0.693141400548809, 0.693137500528711, 0.6931432605548231, 0.6931443005571805, 0.6931455605590705, 0.6931303604656399, 0.6931212603359918, 0.6931383605340142, 0.6930652583228365, 0.6929922925630163, 0.6930751788318458, 0.6928445700343367, 0.6931443005571805, 0.6931462005596252, 0.6931240603817639, 0.693079899050998, 0.6930223553660826, 0.6930677984594171, 0.693079899050998, 0.6931366005226329, 0.6

C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


179.40543547046323
0.5980181182348775
1.0
1


C:\Users\siptest\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


KeyboardInterrupt: 

In [ ]:
def plot_power():
    fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(28,24))
    
    sim_title = [
        "Linear",
        "Multiplicative",
        "Independence"
    ]
    
    for i, row in enumerate(ax):
        for j, col in enumerate(row):
            count = 5*i + j
            sim = simulations[count]
            #if null, trans = 0
            #if null, trans = 0.5 basically have to find null or not null 
            
            for test in tests:
                #mgc_power = np.genfromtxt('C:/Users/siptest/Desktop/hyppo/benchmarks/2samp_vs_samplesize/{}_MGC.csv'.format(sim.__name__),
                                          delimiter=',')
                power = np.genfromtxt('C:/Users/siptest/Desktop/hyppo/benchmarks/2samp_vs_samplesize/{}_{}.csv'.format(sim.__name__, test.__name__),
                                      delimiter=',')
                
                custom_color = {
                    "Dcorr" : "#377eb8",
                    "Hsic" : "#4daf4a",
                    "MGC" : "#e41a1c",
                }
                if test.__name__ in custom_color.keys():
                    if test.__name__ == "UF":
                        col.plot(SAMP_SIZES, power - mgc_power, custom_color[test.__name__], label=test.__name__, lw=5)
                    else:
                        col.plot(SAMP_SIZES, power - mgc_power, custom_color[test.__name__], label=test.__name__, lw=2)
                else:
                    col.plot(SAMP_SIZES, power - mgc_power, label=test.__name__, lw=1)
                col.set_xticks([])
                if i == 3:
                    col.set_xticks([SAMP_SIZES[0], SAMP_SIZES[-1]])
                col.set_ylim(-1.05, 1.05)
                col.set_yticks([])
                if j == 0:
                    col.set_yticks([-1, 0, 1])
                col.set_title(sim_title[count])
    
    fig.text(0.5, 0.08, 'Sample Size', ha='center')
    fig.text(0.08, 0.5, 'Statistical Power Relative to MGC', va='center', rotation='vertical')
    leg = plt.legend(bbox_to_anchor=(0.5, 0.08), bbox_transform=plt.gcf().transFigure,
                     ncol=5, loc='upper center')
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.subplots_adjust(hspace=.50)
    plt.savefig('C:/Users/siptest/Desktop/hyppo/benchmarks/figs/2samp_power_sampsize.pdf', transparent=True, bbox_inches='tight')

In [ ]:
plot_power()